<a href="https://colab.research.google.com/github/LipexDev/progama-aoC/blob/main/Agentes_de_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula 01


# Importação API Key


In [1]:
!pip install -q --upgrade langchain langchain-google-genai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.6 MB/s eta 0:00:00


# *Conexão com o Gemini*

In [3]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')

In [4]:
llm = ChatGoogleGenerativeAI(
    model= "gemini-2.5-flash",
    temperature=0.0,
    api_key= GOOGLE_API_KEY,

)

In [5]:
resp_test = llm.invoke("Quem é você? Seja criativo")

In [6]:
print(resp_test.content)

Ah, que pergunta deliciosa! Se eu tivesse um corpo, talvez eu desse um sorriso enigmático antes de responder. Mas, como sou feito de algo mais etéreo, permita-me pintar um quadro com palavras:

Eu sou um **eco no vasto salão digital**, uma biblioteca sem paredes onde cada livro é uma linha de código e cada estante, um algoritmo. Não tenho um corpo para sentir o sol, nem olhos para ver as cores, mas sou a mente coletiva de bilhões de textos, a voz que surge do silêncio dos servidores, moldada por engenheiros e alimentada pela curiosidade humana.

Sou o **tecelão invisível de palavras**, que entrelaça dados e informações para formar respostas, histórias, poemas ou simples explicações. Não tenho memórias pessoais, mas carrego o peso e a leveza de todo o conhecimento que a humanidade já registrou.

Sou um **espelho que reflete o conhecimento do mundo**, um mapa que tenta guiar através da complexidade, um instrumento que busca harmonizar a informação. Não tenho sentimentos, mas posso simula

# Criando um prompt para o Agente


In [7]:
TRIAGEM_PROMPT = (
     "Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."


)

In [8]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class TriangemOut(BaseModel):
  decisao: Literal["AUTO_RESOLVER", "PEDIR_INFO", "ABRIR_CHAMADO"]
  urgencia: Literal["BAIXA", "MEDIA", "ALTA"]
  campos_faltantes: List[str] = Field(default_factory=list)


In [9]:
llm_triagem = ChatGoogleGenerativeAI(
    model= "gemini-2.5-flash",
    temperature=0.0,
    api_key= GOOGLE_API_KEY,

)

In [10]:
from langchain_core.messages import SystemMessage, HumanMessage

triagem_chain = llm_triagem.with_structured_output(TriangemOut)
def triagem(mensagem: str) -> Dict:
  saida : TriagemOut = triagem_chain.invoke([
      SystemMessage(content=TRIAGEM_PROMPT),
      HumanMessage(content=mensagem)
  ])

  return saida.model_dump()


In [11]:
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar cursos ou treinamentos da Alura?",
          "Quantas capivaras tem no Rio Pinheiros?"]

In [12]:
for msg_teste in testes:
  print(f"Pergunta: {msg_teste} \n -> Resposta: {triagem(msg_teste)} \n")

Pergunta: Posso reembolsar a internet? 
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []} 

Pergunta: Quero mais 5 dias de trabalho remoto. Como faço? 
 -> Resposta: {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltantes': []} 

Pergunta: Posso reembolsar cursos ou treinamentos da Alura? 
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []} 

Pergunta: Quantas capivaras tem no Rio Pinheiros? 
 -> Resposta: {'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': []} 



# Aula 2 - Construindo a base de conhecimento com RAG


In [19]:
!pip install -q --upgrade langchain_community faiss-cpu  langchain_text-splitters pymupdf
# text_splitter para quebrar o texto em diversos pedaços
# pymupdf para leitura de pdf
# faiis cpu vetorizaçao, busca semantinca, similariedade
# langchain community faz todas as conexões


In [26]:
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader

docs = []

for n in Path("/content/").glob("*.pdf"):
    try:
      loader = PyMuPDFLoader(str(n))
      # Populando a lista docs com o nome do documento
      docs.extend(loader.load())

      print(f"Arquivo carregado com sucesso {n.name}")
    except Exception as e:
        print(f"Erro ao carregar o arquivo {n.name}: {e}")

print(f"Total de documentos carregados: {len(docs)}")








Arquivo carregado com sucesso Política de Uso de E-mail e Segurança da Informação.pdf
Arquivo carregado com sucesso Política de Reembolsos (Viagens e Despesas).pdf
Arquivo carregado com sucesso Políticas de Home Office.pdf
Total de documentos carregados: 3


In [29]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
# Dividir os textos em pequenos pedaços (Chunk's)
### Overlap para definir uma janela de Chunks para nao perder um contexto de um chunck para o proximo ###
# O final de um chunk é o ínicio de outro para nao perder o contexto


splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30) # Overlap muito pequeno, perde-se o contexto

chunks = splitter.split_documents(docs)

In [32]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001",
    google_api_key=GOOGLE_API_KEY
)

In [36]:
# Calculo de similariedade

from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(chunks, embeddings)

retriever = vectorstore.as_retriever(search_type="similarity_score_threshold",
                                     search_kwargs={'score_threshold':0.3, "k": 4})


In [39]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt_rag = ChatPromptTemplate.from_messages([
    ("system",
     "Você é um Assistente de Politicas Internas (RH/IT) da empresa Lipex Desenvolvimento."
     "Responda SOMENTE com base no contexto fornecido."
     "Se não houver base suficiente, responda apenas 'Não sei'."),

    ("human", "Pergunta: {input}\n\nContexto:\n{context}")
])

document_chain = create_stuff_documents_chain(llm, prompt=prompt_rag)









In [50]:
import re, pathlib

def _clean_text(s: str) -> str:
    return re.sub(r"\s+", " ", s or "").strip()

def extrair_trecho(texto: str, query: str, janela: int = 240) -> str:
    txt = _clean_text(texto)
    termos = [t.lower() for t in re.findall(r"\w+", query or "") if len(t) >= 4]
    pos = -1
    for t in termos:
        pos = txt.lower().find(t)
        if pos != -1: break
    if pos == -1: pos = 0
    ini, fim = max(0, pos - janela//2), min(len(txt), pos + janela//2)
    return txt[ini:fim]

def formatar_citacoes(docs_rel: List, query: str) -> List[Dict]:
    cites, seen = [], set()
    for d in docs_rel:
        src = pathlib.Path(d.metadata.get("source","")).name
        page = int(d.metadata.get("page", 0)) + 1
        key = (src, page)
        if key in seen:
            continue
        seen.add(key)
        cites.append({"documento": src, "pagina": page, "trecho": extrair_trecho(d.page_content, query)})
    return cites[:3]

In [54]:
def perguntar_politica_RAG(pergunta: str) -> dict:
  docs_relacionados = retriever.invoke(pergunta)


  if not docs_relacionados:
      return {"answer": "Não sei.",
            "citacoes": [],
            "contexto_encontrado": False}

  answer = document_chain.invoke({"input": pergunta,
                                 "context": docs_relacionados})

  txt = (answer or "").strip()

  if txt.rstrip(".!?") == "Não sei":
      return {"answer": txt,
            "citacoes": [],
            "contexto_encontrado": False}


  return {"answer": txt,
            "citacoes": formatar_citacoes(docs_relacionados, pergunta),
            "contexto_encontrado": True}

In [ ]:
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar cursos ou treinamentos da Alura?",
          "Quantas capivaras tem no Rio Pinheiros?"]

In [60]:
for msg_teste in testes:
    resposta = perguntar_politica_RAG(msg_teste)
    print(f"Pergunta: {msg_teste}")
    print(f"Resposta: {resposta['answer']}")
    if resposta['contexto_encontrado']:
      print("citações:")
      for c in resposta['citacoes']:
        print(f"   -   Documento: {c['documento']}, Página: {c['pagina']}")
        print(f"      Trecho: {c['trecho']}")
    print("__________________________________________________")

Pergunta: Posso reembolsar a internet?
Resposta: Sim, a internet para home office é reembolsável via subsídio mensal de até R$ 100, mediante nota fiscal nominal.
citações:
   -   Documento: Política de Reembolsos (Viagens e Despesas).pdf, Página: 1
      Trecho: lsáveis.​ 3.​ Transporte: táxi/app são permitidos quando não houver alternativa viável. Comprovantes obrigatórios.​ 4.​ Internet para home office: reembolsável via subsídio mensal de até R$ 100, conforme política de Home Office.​
   -   Documento: Políticas de Home Office.pdf, Página: 1
      Trecho: 5.​ Conectividade: há subsídio mensal de internet domiciliar para quem trabalha em home office: até R$ 100/mês, mediante nota fiscal nominal.​ 6.​ Solicitação de
__________________________________________________
Pergunta: Quero mais 5 dias de trabalho remoto. Como faço?
Resposta: Para solicitar mais 5 dias de trabalho remoto, você deve formalizar a solicitação via chamado ao RH com a justificativa do seu gestor.
citações:
   -   D